In [1]:
import os 
import sys 
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection
from colbert import Indexer, Searcher
from datasets import load_dataset

ds = load_dataset("wikimedia/wikipedia", "20231101.en")



/Users/memento/Documents/Studium/Master/IR/IR_Praktikum/.env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 6407814
    })
})

In [ ]:
ds['train']['text'][:3]

KeyboardInterrupt: 

In [2]:
from tqdm import tqdm
for i, entry in tqdm(enumerate(iter(ds['train']))):
    if i>10:
        break
    print(entry['title'])

11it [00:00, 2381.41it/s]

Anarchism
Albedo
A
Alabama
Achilles
Abraham Lincoln
Aristotle
An American in Paris
Academy Award for Best Production Design
Academy Awards
Actrius


In [ ]:

'''
Datasets I want to have in the dataset: 
- d_id, title(label), text(erster absatz bis zum ersten \n\n)
- disambiguation dataset
'''

import pandas as pd
from tqdm import tqdm
import re
import os

def clean_label(label):
    pattern = r'\([^)]*\)'
    # Replace all matches with an empty string
    result = re.sub(pattern, '', label)
    return result.strip()  # Added strip() to remove any trailing spaces

def clean_text(text):
    if '\n\n' in text:
        text_cut = text.split('\n\n')[0]
    else:
        text_cut = text
    return text_cut

def is_disambiguation(label):
    # Check if the label contains "(disambiguation)"
    return "(disambiguation)" in label

def format_doc_id(index):
    # Format with 7 digits (since there are 6,352,465 docs total)
    return f"d_{index:07d}"

# Initialize empty DataFrames
doc_df = pd.DataFrame(columns=['d_id', 'label', 'text'])
disambiguation_df = pd.DataFrame(columns=['d_id', 'label', 'text'])

# Lists to store rows for each DataFrame (more efficient than append)
doc_rows = []
disambiguation_rows = []

for i, entry in tqdm(enumerate(iter(ds['train']))):
    # Check if it's a disambiguation page before cleaning
    doc_id = format_doc_id(i + 1)
    if is_disambiguation(entry['title']):
        disambiguation_rows.append({'d_id': doc_id, 'label': entry['title'], 'text': entry['text']})
    else:
        # For regular documents
        cleaned_label = clean_label(entry['title'])
        cleaned_text = clean_text(entry['text'])
        doc_rows.append({'d_id': doc_id, 'label': cleaned_label, 'text': cleaned_text})

    # Create DataFrames from the collected rows (much more efficient)
    
doc_df = pd.DataFrame(doc_rows)
disambiguation_df = pd.DataFrame(disambiguation_rows)

# Print some stats to verify
print(f"Regular documents: {len(doc_df)}")
print(f"Disambiguation documents: {len(disambiguation_df)}")
pwd = os.getcwd()
datapath = os.path.join(pwd, '../../data/wikipedia')
os.makedirs(datapath, exist_ok=True)
doc_df.to_pickle(path=os.path.join(datapath, 'wikipedia-text-data-no-disambiguation.pkl.gzip'), compression='gzip')
disambiguation_df.to_pickle(path=os.path.join(datapath, 'wikipedia-disambiguation-data.pkl.gzip'), compression='gzip')

6407814it [03:16, 32624.71it/s]


Regular documents: 6352465
Disambiguation documents: 55349


In [4]:
i = 0
while(True):
    if (i*500000)-1 > 6352465:
        break
    wd = os.getcwd()
    datapath = os.path.join(pwd, '../../data/wikipedia/split-data-no-disambiguation')
    os.makedirs(datapath, exist_ok=True)    
    
    doc_df[i*500000:((i+1)*500000)-1].to_pickle(path=os.path.join(datapath, f'wikipedia-text-data-no-disambiguation_{i}.pkl.gzip'), compression='gzip')
    i=i+1

ds['train']['text'][:3]

# Add test dataset with just articals containing
+ Jaguar
+ Hammer
+ Coke
+ Pope
+ Comic
+ Drink
+ Cat
+ Dwarf
+ Game
+ Texas

# Add Query Dataset for all disambiguations

In [1]:
disambiguation_df[:20]

NameError: name 'disambiguation_df' is not defined